<a href="https://colab.research.google.com/github/akurniawan/ICS5110-final/blob/main/01-Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import math
import nltk
import numpy as np
import spacy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
review_df = pd.read_csv("test.csv", delimiter="\t")

In [ ]:
review_df.head()

,review_id,stars,useful,text
0,xQY8N_XvtGbearJ5X4QryQ,2.0,5,"As someone who has worked with many museums, I..."
1,UmFMZ8PyXZTY2QcwzsfQYA,1.0,1,I am actually horrified this place is still in...
2,LG2ZaYiOgpr2DK_90pYjNw,5.0,1,I love Deagan's. I do. I really do. The atmosp...
3,i6g_oA9Yf9Y31qt0wibXpw,1.0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."
4,6TdNDKywdbjoTkizeMce8A,4.0,0,"Oh happy day, finally have a Canes near my cas..."


In [ ]:
review_df.count()

review_id    8021122
stars        8021122
useful       8021122
text         8021120
dtype: int64

In [ ]:
nlp = spacy.load("en")

def lemmatize(text):
    doc = nlp(text)
    lem = []
    for token in doc:
      lem.append(token.lemma_)
    return " ".join(lem)

def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

def preprocessing(text):
    text = str(text).strip().lower()
    text = remove_special_characters(text)
    text = lemmatize(text)
    return text

#Apply function on review column
review_df["text"] = review_df["text"].apply(preprocessing)
review_df["unigram"] = review_df["text"].apply(lambda x: x.split(" "))

NameError: name 'df' is not defined

In [ ]:
review_df.head()